In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, year, size, split, substring
)

import os
from geopy.geocoders import Nominatim

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('User Locations')
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-2]) 

data_input_path = os.path.join(project_path, 'data/users')

data_output_path = os.path.join(project_path, 'data/locations.csv')

In [ ]:
usersDF = (
    spark
    .read
    .option('path', data_input_path)
    .load()
)

In [ ]:
locationsDF = (
    usersDF
    .filter(col('location').isNotNull())
    .select('location')
    .distinct()
)

In [ ]:
locationsDF.show(n=5)

In [ ]:
locations_local = [r['location'] for r in locationsDF.collect()]

In [ ]:
len(locations_local)

In [ ]:
geolocator = Nominatim(user_agent="User Locations")

In [ ]:
result = []

In [ ]:
def get_coordinates(location_name):
    try:
        location = geolocator.geocode(location_name)
        return (location_name, location.latitude, location.longitude)
    except:
        return None

In [ ]:
i = 0
for loc in locations_local:
    coords = get_coordinates(loc)
    print(i)
    if (coords):
        result.append(coords)
    i = i + 1

In [ ]:
len(result)

In [ ]:
coordsDF = spark.createDataFrame(result, ['name', 'latitude', 'longitude'])

In [ ]:
coordsDF.show()

In [ ]:
pandasDF = coordsDF.toPandas()

In [ ]:
pandasDF.to_csv(path_or_buf=data_output_path, header=True, index=False)